In [1]:
"""
TRUE BASE GENERATION SYSTEM
No external knowledge sources - pure LLM generation from text only
All external APIs, Knowledge Graphs, and Location enrichment DISABLED
"""

import os
import re
import time
import logging
from typing import List, Dict, Any
import tiktoken
from dotenv import load_dotenv

# Only essential imports - no external knowledge sources
from langchain_anthropic import ChatAnthropic

from langchain.schema import HumanMessage

# Configuration
INPUT_TEXT_FILE = "part_aa"
OUTPUT_BASE_TTL = 'extracted_events_base_generation_ckaude.ttl'

# TRUE BASE GENERATION FLAGS - All external knowledge DISABLED
RAG_ENABLED = False
KNOWLEDGE_GRAPHS_ENABLED = False
LOCATION_ENRICHMENT_ENABLED = False
EXTERNAL_APIS_ENABLED = False

# Token limits
MAX_TOKENS_PER_REQUEST = 100000
CHUNK_OVERLAP = 200

# Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class TextChunker:
    """Handles text chunking to manage token limits"""
    
    def __init__(self, model_name: str = "claude-sonnet-4-20250514"):
        self.model_name = model_name
    
    def count_tokens(self, text: str) -> int:
        """Approximate token count for Claude (roughly 4 chars per token)"""
        return len(text) // 4
    
    def chunk_text_by_sentences(self, text: str, max_tokens: int = 15000) -> List[str]:
        """Chunk text by sentences to maintain coherence"""
        sentences = re.split(r'[.!?]+', text)
        chunks = []
        current_chunk = ""
        
        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue
                
            test_chunk = current_chunk + " " + sentence if current_chunk else sentence
            
            if self.count_tokens(test_chunk) > max_tokens and current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = sentence
            else:
                current_chunk = test_chunk
        
        if current_chunk.strip():
            chunks.append(current_chunk.strip())
        
        return chunks

class TrueBaseGenerationSystem:
    """TRUE Base Generation System - No external knowledge sources"""
    
    def __init__(self):
        self.chunker = TextChunker()
        self.stats = {
            'chunks_processed': 0,
            'events_extracted': 0,
            'external_api_calls': 0,  # This will stay 0
            'knowledge_sources_used': 0,  # This will stay 0
        }
        logger.info("TRUE BASE GENERATION SYSTEM INITIALIZED")
        logger.info("❌ Knowledge Graphs: DISABLED")
        logger.info("❌ Location Enrichment: DISABLED") 
        logger.info("❌ External APIs: DISABLED")
        logger.info("❌ RAG Text Retrieval: DISABLED")
        logger.info("✅ Pure LLM Generation: ENABLED")
    
    def extract_basic_entities_from_text_only(self, text: str) -> List[str]:
        """Extract entities using ONLY pattern matching - no external validation"""
        # Simple pattern matching - no external knowledge validation
        pattern = r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b'
        matches = re.findall(pattern, text)
        
        # Basic stopword filtering - no external knowledge
        stop_words = {
            'The', 'This', 'That', 'These', 'Those', 'And', 'But', 'Or', 'So', 'If', 
            'When', 'Where', 'Who', 'What', 'How', 'Why', 'All', 'Some', 'Many', 
            'First', 'Second', 'Third', 'Last', 'Next', 'Before', 'After', 'During'
        }
        
        filtered_entities = []
        for entity in matches:
            entity = entity.strip()
            if (entity not in stop_words and len(entity) > 2 and not entity.isdigit()):
                filtered_entities.append(entity)
        
        # Remove duplicates - keep first 10 to avoid overwhelming the prompt
        seen = set()
        unique_entities = []
        for entity in filtered_entities:
            if entity.lower() not in seen:
                seen.add(entity.lower())
                unique_entities.append(entity)
        
        return unique_entities[:10]  # Limited to avoid prompt bloat
    
    def extract_basic_locations_from_text_only(self, text: str) -> List[str]:
        """Extract locations using ONLY pattern matching - no coordinate lookup"""
        # Simple pattern matching for potential locations - no external validation
        location_patterns = [
            r'\b[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*(?:\s+(?:City|County|State|Province|Country|Region|Island|Bay|Sea|Ocean|River|Mountain|Valley|Desert))\b',
            r'\b(?:Mount|Lake|River|Cape|Fort|Port|Saint|St\.)\s+[A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+)*\b',
            r'\b[A-Z][a-zA-Z]{2,}(?:\s+[A-Z][a-zA-Z]{2,})*\b'
        ]
        
        locations = []
        for pattern in location_patterns:
            matches = re.findall(pattern, text)
            locations.extend(matches)
        
        # Basic filtering - no external knowledge
        location_stopwords = {
            'The', 'This', 'That', 'And', 'But', 'Or', 'So', 'If', 'When', 'Where',
            'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
            'September', 'October', 'November', 'December'
        }
        
        filtered_locations = []
        for loc in locations:
            loc = loc.strip()
            if (loc not in location_stopwords and len(loc) > 2 and not loc.isdigit()):
                filtered_locations.append(loc)
        
        return list(set(filtered_locations))[:5]  # Limited to avoid prompt bloat
    
    def process_chunk_true_base(self, chunk: str, chunk_num: int, llm) -> str:
        """Process chunk with TRUE base generation - no external knowledge"""
        logger.info(f"Processing chunk {chunk_num} ({len(chunk)} chars) - TRUE BASE GENERATION")
        
        # ONLY extract entities/locations from text patterns - NO external validation
        entities = self.extract_basic_entities_from_text_only(chunk)
        locations = self.extract_basic_locations_from_text_only(chunk)
        
        logger.info(f"Found entities (text-only): {entities[:3]}...")
        logger.info(f"Found locations (text-only): {locations[:3]}...")
        
        if not entities and not locations:
            logger.info(f"No entities or locations found in chunk {chunk_num}")
            return ""
        
        # TRUE BASE GENERATION WITH INFERENCE PROMPT - SAME FORMAT AS ENHANCED SYSTEMS
        base_prompt = f"""You are extracting historical events from text using ONLY the information provided in the text chunk. Do not use external knowledge sources, but you CAN and SHOULD make reasonable inferences from the text.

TEXT CHUNK {chunk_num} TO ANALYZE:
{chunk}

ENTITIES FOUND IN TEXT: {', '.join(entities) if entities else 'None'}
LOCATIONS FOUND IN TEXT: {', '.join(locations) if locations else 'None'}

TASK: Extract historical events mentioned in this text chunk using the text information and making REASONABLE INFERENCES.

REQUIREMENTS:
1. Extract ONLY events explicitly mentioned in the text chunk
2. Use information directly stated in the text
3. MAKE REASONABLE INFERENCES from context clues in the text
4. If you can reasonably infer coordinates, countries, regions from textual context, DO IT
5. Include ALL these properties for each event:
   - ste:hasType (description of event, enhanced with context)
   - ste:hasAgent (who caused/led the event, with inferred roles)
   - ste:hasTime (when it happened, with inferred specificity)
   - ste:hasLocation (location name from text)
   - ste:hasLatitude (infer approximate coordinates if you can from text context)
   - ste:hasLongitude (infer approximate coordinates if you can from text context)
   - ste:hasCountry (infer country from textual geographic context)
   - ste:hasRegion (infer region from textual geographic context)
   - ste:hasLocationSource "inferred" (if you made geographic inferences)
   - ste:hasResult (outcome, enhanced with contextual inference)

INFERENCE GUIDELINES:
- If text mentions "Athens", infer it's in Greece, approximate coordinates
- If text mentions "Sicily", infer it's in Italy, Mediterranean coordinates  
- If text mentions "Sparta/Lacedaemon", infer Peloponnese, Greece
- If you know from context clues what geographic region events occurred in, infer coordinates
- If someone is called "King", infer royal title
- If text implies timeframes, infer more specific dates
- If outcomes are implied, infer likely results

Output format (do not include prefixes):
```turtle
ste:Event{chunk_num}_1 a ste:Event ;
    ste:hasType "specific event type inferred from context" ;
    ste:hasAgent "person/group with inferred roles" ;
    ste:hasTime "time period with inferred specificity" ;
    ste:hasLocation "location name from text" ;
    ste:hasLatitude "37.9838" ;
    ste:hasLongitude "23.7275" ;
    ste:hasCountry "Greece" ;
    ste:hasRegion "Attica" ;
    ste:hasLocationSource "inferred" ;
    ste:hasResult "outcome inferred from context" .
```

CRITICAL: 
- Use the SAME output format as enhanced systems for fair comparison
- INFER coordinates, countries, regions if you can reasonably deduce them from text
- Make the events as detailed and specific as possible through inference
- If you truly cannot infer something, then use empty string ""
- The goal is to extract maximum information through text analysis and inference

If no clear historical events are mentioned in the text, return empty.
"""
        
        try:
            response = llm.invoke([HumanMessage(content=base_prompt)])
            turtle_output = self.clean_turtle(response.content)
            
            if turtle_output:
                self.stats['chunks_processed'] += 1
                # Count events by counting "ste:Event" occurrences
                event_count = turtle_output.count('ste:Event')
                self.stats['events_extracted'] += event_count
                logger.info(f"Generated {event_count} events from chunk {chunk_num} (base generation)")
            
            return turtle_output
            
        except Exception as e:
            logger.error(f"Error processing chunk {chunk_num}: {e}")
            return ""
    
    def clean_turtle(self, raw_output: str) -> str:
        """Clean turtle output"""
        # Extract turtle code block if present
        m = re.search(r"```(?:turtle)?\s*(.*?)```", raw_output, re.DOTALL | re.IGNORECASE)
        if m:
            return m.group(1).strip()
        
        # If no code block, try to extract turtle-like lines
        lines = raw_output.strip().split('\n')
        turtle_lines = []
        for line in lines:
            stripped = line.strip()
            if (stripped.startswith('@') or stripped.startswith('<') or 
                stripped.startswith(':') or stripped.startswith('ste:') or 
                stripped.startswith('a ') or ':' in stripped or stripped == ''):
                turtle_lines.append(line)
        
        return '\n'.join(turtle_lines)

# Utility functions
def load_api_key():
    """Load Anthropic API key"""
    load_dotenv()
    api_key = os.getenv("ANTHROPIC_API_KEY")
    if not api_key:
        print("Error: ANTHROPIC_API_KEY not found")
        return None
    print("✅ Anthropic API Key loaded successfully.")
    return api_key

def load_text_from_file(filepath: str) -> str:
    """Load text from file"""
    if not os.path.isfile(filepath):
        print(f"❌ File not found: {filepath}")
        return ""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read().strip()
        print(f"✅ Loaded text from {filepath}")
        return text
    except Exception as e:
        print(f"❌ Error reading file {filepath}: {e}")
        return ""

def initialize_llm(api_key: str):
    """Initialize LLM"""
    if not api_key:
        return None
    try:
        llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0, anthropic_api_key=api_key)
        print("✅ LLM initialized successfully.")
        return llm
    except Exception as e:
        print(f"❌ Error initializing LLM: {e}")
        return None

def main():
    """Main function - TRUE BASE GENERATION ONLY"""
    print("🚀 Starting TRUE BASE GENERATION SYSTEM")
    print("="*60)
    print("❌ Knowledge Graphs: DISABLED")
    print("❌ Location Coordinate Lookup: DISABLED") 
    print("❌ External APIs: DISABLED")
    print("❌ RAG Text Retrieval: DISABLED")
    print("❌ All External Knowledge Sources: DISABLED")
    print("✅ Pure LLM Generation from Text Only: ENABLED")
    print("="*60)
    
    api_key = load_api_key()
    if not api_key:
        return
    
    domain_text = load_text_from_file(INPUT_TEXT_FILE)
    if not domain_text:
        print("⚠️  No input file found, using sample text")
        domain_text = """The Battle of Salamis was a decisive naval battle in 480 BC. 
        Themistocles led the Greek fleet to victory over the Persians commanded by Xerxes. 
        This victory established Greek naval supremacy in the Aegean Sea."""
    else:
        print(f"📄 Using text from {INPUT_TEXT_FILE}")
        print(f"📝 Text length: {len(domain_text)} characters")
    
    base_system = TrueBaseGenerationSystem()
    llm = initialize_llm(api_key)
    
    if not llm:
        return
    
    token_count = base_system.chunker.count_tokens(domain_text)
    print(f"🔢 Total tokens in text: {token_count:,}")
    
    if token_count > 15000:
        print("📊 Text is large, chunking into smaller pieces...")
        chunks = base_system.chunker.chunk_text_by_sentences(domain_text, max_tokens=15000)
        print(f"📄 Created {len(chunks)} chunks")
    else:
        print("📄 Text is small enough to process as single chunk")
        chunks = [domain_text]
    
    # Process chunks with TRUE base generation
    all_turtle_outputs = []
    
    print("\n🔄 Processing chunks with TRUE BASE GENERATION...")
    for i, chunk in enumerate(chunks, 1):
        print(f"\n🔄 Processing chunk {i}/{len(chunks)}...")
        
        turtle_output = base_system.process_chunk_true_base(chunk, i, llm)
        if turtle_output:
            all_turtle_outputs.append(turtle_output)
        
        if i < len(chunks):
            time.sleep(0.5)  # Brief pause between chunks
    
    # Save RDF output
    if all_turtle_outputs:
        prefixes = """@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

"""
        
        final_output = prefixes + "# TRUE BASE GENERATION - No External Knowledge Sources\n" + "\n\n".join(all_turtle_outputs)
        
        with open(OUTPUT_BASE_TTL, 'w', encoding='utf-8') as f:
            f.write(final_output)
        
        print(f"\n✅ Saved TRUE BASE GENERATION RDF to {OUTPUT_BASE_TTL}")
        print(f"📊 TRUE BASE GENERATION Statistics:")
        print(f"   - Generation Mode: PURE BASE (No External Knowledge)")
        print(f"   - Total chunks processed: {len(chunks)}")
        print(f"   - Successful chunks: {len(all_turtle_outputs)}")
        print(f"   - Events extracted: {base_system.stats['events_extracted']}")
        print(f"   - External API calls: {base_system.stats['external_api_calls']} (should be 0)")
        print(f"   - Knowledge sources used: {base_system.stats['knowledge_sources_used']} (should be 0)")
        print(f"   - Knowledge Graph queries: 0 (DISABLED)")
        print(f"   - Location coordinate lookups: 0 (DISABLED)")
        print(f"   - RAG text retrievals: 0 (DISABLED)")
        
        print(f"\n📝 Sample of TRUE BASE GENERATION RDF:")
        print("="*60)
        print(final_output[:800] + "..." if len(final_output) > 800 else final_output)
        print("="*60)
        
        print(f"\n🎯 VERIFICATION:")
        print(f"   ✅ No external knowledge was used")
        print(f"   ✅ No API calls were made")
        print(f"   ✅ Only text-based pattern matching was used")
        print(f"   ✅ LLM used only information from the input text")
        
    else:
        print("❌ No events were extracted from any chunks")
        print("💡 This might be because:")
        print("   - The text doesn't contain clear historical events")
        print("   - The base generation approach is more conservative")
        print("   - Without external knowledge, fewer entities were recognized")
    
    print(f"\n🎉 TRUE BASE GENERATION complete!")
    print(f"📄 Output file: {OUTPUT_BASE_TTL}")
    print(f"🔍 This output contains ONLY information from your text, no external knowledge")

if __name__ == '__main__':
    main()

2025-05-29 07:26:58,735 - INFO - TRUE BASE GENERATION SYSTEM INITIALIZED
2025-05-29 07:26:58,736 - INFO - ❌ Knowledge Graphs: DISABLED
2025-05-29 07:26:58,736 - INFO - ❌ Location Enrichment: DISABLED
2025-05-29 07:26:58,736 - INFO - ❌ External APIs: DISABLED
2025-05-29 07:26:58,736 - INFO - ❌ RAG Text Retrieval: DISABLED
2025-05-29 07:26:58,736 - INFO - ✅ Pure LLM Generation: ENABLED
2025-05-29 07:26:58,766 - INFO - Processing chunk 1 (59851 chars) - TRUE BASE GENERATION
2025-05-29 07:26:58,770 - INFO - Found entities (text-only): ['The Project Gutenberg', 'The History', 'Peloponnesian War']...
2025-05-29 07:26:58,770 - INFO - Found locations (text-only): ['Peloponnesian War', 'START', 'Achaeans']...


🚀 Starting TRUE BASE GENERATION SYSTEM
❌ Knowledge Graphs: DISABLED
❌ Location Coordinate Lookup: DISABLED
❌ External APIs: DISABLED
❌ RAG Text Retrieval: DISABLED
❌ All External Knowledge Sources: DISABLED
✅ Pure LLM Generation from Text Only: ENABLED
✅ Anthropic API Key loaded successfully.
✅ Loaded text from part_aa
📄 Using text from part_aa
📝 Text length: 398568 characters
✅ LLM initialized successfully.
🔢 Total tokens in text: 99,642
📊 Text is large, chunking into smaller pieces...
📄 Created 7 chunks

🔄 Processing chunks with TRUE BASE GENERATION...

🔄 Processing chunk 1/7...


2025-05-29 07:27:13,877 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-05-29 07:27:13,903 - INFO - Generated 12 events from chunk 1 (base generation)
2025-05-29 07:27:14,409 - INFO - Processing chunk 2 (59939 chars) - TRUE BASE GENERATION
2025-05-29 07:27:14,424 - INFO - Found entities (text-only): ['They', 'Chimerium', 'Thesprotis']...
2025-05-29 07:27:14,425 - INFO - Found locations (text-only): ['Surely', 'Some', 'Peloponnesian War']...



🔄 Processing chunk 2/7...


2025-05-29 07:27:32,504 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-05-29 07:27:32,509 - INFO - Generated 12 events from chunk 2 (base generation)
2025-05-29 07:27:33,012 - INFO - Processing chunk 3 (59850 chars) - TRUE BASE GENERATION
2025-05-29 07:27:33,029 - INFO - Found entities (text-only): ['Manifold', 'Hellenes', 'Sparta']...
2025-05-29 07:27:33,030 - INFO - Found locations (text-only): ['Hellespont', 'Dorians', 'Ithome Most']...


...


🔄 Processing chunk 6/7...


2025-05-29 07:28:42,655 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-05-29 07:28:42,659 - INFO - Generated 10 events from chunk 6 (base generation)
2025-05-29 07:28:43,165 - INFO - Processing chunk 7 (36927 chars) - TRUE BASE GENERATION
2025-05-29 07:28:43,177 - INFO - Found entities (text-only): ['Mitylenians', 'Peloponnese', 'Mitylene']...
2025-05-29 07:28:43,178 - INFO - Found locations (text-only): ['Ephesus', 'Chians', 'Ionian']...



🔄 Processing chunk 7/7...


2025-05-29 07:28:59,089 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-05-29 07:28:59,093 - INFO - Generated 10 events from chunk 7 (base generation)



✅ Saved TRUE BASE GENERATION RDF to extracted_events_base_generation_ckaude.ttl
📊 TRUE BASE GENERATION Statistics:
   - Generation Mode: PURE BASE (No External Knowledge)
   - Total chunks processed: 7
   - Successful chunks: 7
   - Events extracted: 78
   - External API calls: 0 (should be 0)
   - Knowledge sources used: 0 (should be 0)
   - Knowledge Graph queries: 0 (DISABLED)
   - Location coordinate lookups: 0 (DISABLED)
   - RAG text retrievals: 0 (DISABLED)

📝 Sample of TRUE BASE GENERATION RDF:
@prefix ste: <http://www.example.org/ste#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# TRUE BASE GENERATION - No External Knowledge Sources
ste:Event1_1 a ste:Event ;
    ste:hasType "Commencement of the Peloponnesian War between Athens and Sparta" ;
    ste:hasAgent "Athenians and Peloponnesians led by their respective city-states" ;
    ste:hasTime "431 BC" 